# Important:

This notebook contains code and scripts to fix the files we're using for validation and testing. Don't mess with it!

The files are split based on the timestep count, not the number of files.

In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [2]:
%pwd

'/home/tmarcu/brain2brain/experiments/notebooks'

In [3]:
# Imports
import numpy as np
import pandas as pd
import numpy as np
import time
import string
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
import sys
sys.path.append('../../')
from brain2brain import utils
from brain2brain import generators
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [9]:
all_paths_676 = utils.get_file_paths_from_root(patient_number=676, sort=True)

In [10]:
utils.normalize_dataset(file_paths=all_paths_676, path_list_out="676_bin_norm_2",
                        split_data=True, split_ratio=0.8,
                        output_directory="/projects/HASSON/247/data/binned_conversations/676_bin_norm_2/",
                        file_prefix="bin_norm_2_", binned=True, avg_timestep_count=25)

Splitting data in train, val, test.
Training set normalization.
Normalizing dataset partition
Calculating mean per electrode
Calculating std per electrode
Binning dataset partition.
Normalized file and saved at /projects/HASSON/247/data/binned_conversations/676_bin_norm_2/bin_norm_2_NY676_616_Part1_conversation1/bin_norm_2_NY676_616_Part1_conversation1.npy
Elapsed time: 0.89s 1/54 done
Binning dataset partition.
Normalized file and saved at /projects/HASSON/247/data/binned_conversations/676_bin_norm_2/bin_norm_2_NY676_616_Part1_conversation2/bin_norm_2_NY676_616_Part1_conversation2.npy
Elapsed time: 0.35s 2/54 done
Binning dataset partition.
Normalized file and saved at /projects/HASSON/247/data/binned_conversations/676_bin_norm_2/bin_norm_2_NY676_616_Part1_conversation3/bin_norm_2_NY676_616_Part1_conversation3.npy
Elapsed time: 0.03s 3/54 done
Binning dataset partition.
Normalized file and saved at /projects/HASSON/247/data/binned_conversations/676_bin_norm_2/bin_norm_2_NY676_616_Part

In [4]:
# Read saved paths for training.
saved_paths_676 = utils.get_file_paths("/home/tmarcu/brain2brain/brain2brain/train_676_norm_files_projects.txt")
train_676, val_676 = utils.split_file_paths(saved_paths_676, 0.8)

In [5]:
train_676_generator = generators.FGenerator(file_paths = train_676,
                                            lookback=125, length = 1, delay = 0,
                                            batch_size = 256, sample_period = 1,
                                            electrodes= [0, 1], shuffle = True, debug=False)

In [8]:
train_676_generator[0][0].shape[2]

2

In [13]:
train_676_generator[0][1].shape

(256, 1, 2)

In [6]:
maxt = 0
sumt = 0
for i in range(len(train_676_generator)):
    t0 = time.time()
    x, y = train_676_generator[i]
    t1 = time.time()
    print(f"T:{t1-t0}")
    sumt += (t1 - t0)
    if (t1 - t0) > maxt:
        maxt = t1-t0
print(f"Max: {maxt}")
print(f"Avg: {sumt/len(train_676_generator)}")

T:1.1659440994262695
T:1.1444377899169922
T:1.1483190059661865
T:1.1474039554595947


KeyboardInterrupt: 

In [12]:
print(f"Max: {maxt}")
print(f"Avg: {sumt/len(train_676_generator)}")

Max: 10.227735757827759
Avg: 1.4914311466390007


In [7]:
train_676_generator2 = generators.Generator(file_paths = train_676,
                                            lookback=128, length = 1, delay = 0,
                                            batch_size = 256, sample_period = 1,
                                            num_electrodes=1, shuffle = True)

In [ ]:
maxt = 0
sumt = 0
for i in range(len(train_676_generator2)):
    t0 = time.time()
    x, y = train_676_generator2[i]
    t1 = time.time()
    print(i)
    print(f"T:{t1-t0}")
    sumt += (t1 - t0)
    if (t1 - t0) > maxt:
        maxt = t1-t0
print(f"Max: {maxt}")
print(f"Avg: {sumt/len(train_676_generator2)}")

0
T:372.5089497566223
1
T:355.28351521492004
2
T:355.06990242004395
3
T:326.5501968860626
4
T:374.50031447410583
5
T:351.6825261116028


In [12]:
# Load Files For A Patient
# Split Them Into Train and Test
# Save the File Names
# Load Files for a Patient
files_676 = utils.get_file_paths_from_root(patient_number = 676, shuffle=True)
# Split them into train and test
utils.normalize_files(files_676, output_directory="/projects/HASSON/247/data/normalized-conversations/676-conversations-normalized/")

Original Shape: (30583, 114)

New Shape: (30583, 114)

Normalized file and saved at /projects/HASSON/247/data/normalized-conversations/676-conversations-normalized/norm_NY676_619_Part5-two_conversation2/norm_NY676_619_Part5-two_conversation2.npy
Elapsed time: 0.30s 1/84 done
Original Shape: (388963, 114)

New Shape: (388963, 114)

Normalized file and saved at /projects/HASSON/247/data/normalized-conversations/676-conversations-normalized/norm_NY676_618_Part7_conversation5/norm_NY676_618_Part7_conversation5.npy
Elapsed time: 2.02s 2/84 done
Original Shape: (1623762, 114)

New Shape: (1623762, 114)

Normalized file and saved at /projects/HASSON/247/data/normalized-conversations/676-conversations-normalized/norm_NY676_616_Part1_conversation1/norm_NY676_616_Part1_conversation1.npy
Elapsed time: 5.62s 3/84 done
Original Shape: (1472079, 114)

New Shape: (1472079, 114)

Normalized file and saved at /projects/HASSON/247/data/normalized-conversations/676-conversations-normalized/norm_NY676_620

In [5]:
# Load Normalized Files for a Patient
files_676_norm = utils.get_file_paths_from_dir("/projects/HASSON/247/data/normalized_conversations/676-conversations-normalized/",
                                               sort=False, shuffle=True)
# Split them into train and test
train_676_norm, test_676_norm = utils.split_file_paths(file_paths = files_676_norm, split_ratio=0.8)
print(f"676 Total Timestep Count: {utils.get_total_timestep_count(files_676_norm)}")
print(f"676 TRAIN Total Timestep Count: {utils.get_total_timestep_count(train_676_norm)}")
print(f"676 TEST Total Timestep Count: {utils.get_total_timestep_count(test_676_norm)}")

676 Total Timestep Count: 70518637
676 TRAIN Total Timestep Count: 56372099
676 TEST Total Timestep Count: 14146538


In [6]:
# Save the files
with open('train_676_norm_files_projects.txt', 'w') as filehandle:
    for path in train_676_norm:
        filehandle.write('%s\n' % path)

In [7]:
# Save the files
with open('test_676_norm_files_projects.txt', 'w') as filehandle:
    for path in test_676_norm:
        filehandle.write('%s\n' % path)

In [4]:
# Load Files for a Patient
files_676 = utils.get_file_paths_from_root(patient_number = 676, sort=True, shuffle=True)


In [19]:
files_676[0]

'/projects/HASSON/247/data/676-conversations/NY676_616_Part1_conversation5/NY676_616_Part1_conversation5.npy'

In [8]:
# Split them into train and test
train_676, test_676 = utils.split_file_paths(file_paths = files_676, split_ratio=0.8)


In [10]:
print(f"676 Total Timestep Count: {utils.get_total_timestep_count(files_676)}")
print(f"676 TRAIN Total Timestep Count: {utils.get_total_timestep_count(train_676)}")
print(f"676 TEST Total Timestep Count: {utils.get_total_timestep_count(test_676)}")

676 Total Timestep Count: 70518637
676 TRAIN Total Timestep Count: 55871878
676 TEST Total Timestep Count: 14646759


In [12]:
# # Save the files
# with open('train_676_files.txt', 'w') as filehandle:
#     for path in train_676:
#         filehandle.write('%s\n' % path)

In [13]:
# # # Save the files
# with open('test_676_files.txt', 'w') as filehandle:
#     for path in test_676:
#         filehandle.write('%s\n' % path)

In [47]:
utils.normalize_files(files_676, output_directory="/tmp/tmarcu/normalized-conversations/676-conversations-normalized/")

Normalized file and saved at /tmp/tmarcu/normalized-conversations/676-conversations-normalized/norm_NY676_616_Part1_conversation5/norm_NY676_616_Part1_conversation5.npy
Elapsed time: 1.61s 1/84 done
Normalized file and saved at /tmp/tmarcu/normalized-conversations/676-conversations-normalized/norm_NY676_618_Part7_conversation6/norm_NY676_618_Part7_conversation6.npy
Elapsed time: 1.23s 2/84 done
Normalized file and saved at /tmp/tmarcu/normalized-conversations/676-conversations-normalized/norm_NY676_618_Part4_conversation7/norm_NY676_618_Part4_conversation7.npy
Elapsed time: 0.35s 3/84 done
Normalized file and saved at /tmp/tmarcu/normalized-conversations/676-conversations-normalized/norm_NY676_617_Part1_conversation5/norm_NY676_617_Part1_conversation5.npy
Elapsed time: 0.42s 4/84 done
Normalized file and saved at /tmp/tmarcu/normalized-conversations/676-conversations-normalized/norm_NY676_618_Part4_conversation5/norm_NY676_618_Part4_conversation5.npy
Elapsed time: 2.39s 5/84 done
Norma

In [5]:
with open("/home/tmarcu/brain2brain/brain2brain/train_676_norm_files_projects.txt") as f:
    content = f.readlines()

In [6]:
content[0]

'/projects/HASSON/247/data/normalized-conversations/676-conversations-normalized/norm_NY676_620_Part3-one_conversation4/norm_NY676_620_Part3-one_conversation4.npy\n'